# 1. Producing the data  
In this task, we will implement Apache Kafka producers to simulate real-time data streaming. Spark and parallel data processing should not be used in this section, as we are simulating sensors that often lack processing capabilities.  

1.	Every 5 seconds, load 5 days of weather data from the CSV file. We refer to this as weather5s to explain the tasks; feel free to use your own variable name. You should keep a pointer in the file reading process and advance it per read. The data reading should be in chronological order.
2.	Add the current timestamp (weather_ts) to the weather5s and spread your batch out evenly for 5 seconds for each day. Since the weather data is hourly readings, each day you shall have 24 records (120 records in total for 5 days).
For example, assume you send the records at 2025-01-26 00:00:00 (ISO format: YYYY-MM-DD HH:MM:SS) -> (ts = 1737810000):  
Day 1(records 1-24): ts = 1737810000  
Day 2(records 25-48): ts = 1737810001  
Day 3(records 49-72): ts = 1737810002  
…
3.	Send your batch of weather data to a Kafka topic with an appropriate name.




In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
from datetime import datetime, timedelta
import csv
import io
import json

#configuration
hostip = "192.168.0.6"#"10.192.90.63" #change to your machine IP address

topic = 'weather_data'


def publish_message(producer_instance, topic_name, key, value):
    try:
        print("\nBegin publish")
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: \n' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

def read_csv_dict_chunks(filepath, chunk_size):
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        while True:
            chunk = []
            for _ in range(chunk_size):
                try:
                    row = next(reader)
                    # Add timestamp
                    row["weather_ts"] = datetime.now().isoformat()
                    chunk.append(row)   # each row is already a dict
                except StopIteration:
                    break
            if not chunk:
                break
            yield chunk
            
def read_sorted_csv_in_chunks(filepath, chunk_size=1000, sort_field="timestamp"):
    # Step 1: Read the entire CSV into memory
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        rows = list(reader)

    # Step 2: Sort chronologically by the existing timestamp field
    # (ISO 8601 timestamps can be compared as strings, but we’ll be explicit)
    rows.sort(key=lambda r: datetime.fromisoformat(r[sort_field]))

    # Step 3: Yield chunks with a new timestamp column added at read time
    now = datetime.now()  # one timestamp per chunk
#     print("DEBUG")
#     print(now)
    for i in range(0, len(rows), chunk_size):
        chunk = rows[i:i + chunk_size]        
        for row in chunk:
            new_time = (now + timedelta(seconds=5 * i // chunk_size)).isoformat() # spread out timestamp evenly 1s per 24 records
            row["weather_ts"] = new_time
        yield chunk


def chunk_to_string(chunk):
    return json.dumps(chunk, ensure_ascii=False)


if __name__ == '__main__':   
    producer = connect_kafka_producer()
    for chunk in read_sorted_csv_in_chunks("data/weather.csv", chunk_size=120):
        data = chunk_to_string(chunk)
        
#     for rows in read_csv_dict_chunks("data/weather.csv", chunk_size=120):
#         data = chunk_to_string(rows)
        
        publish_message(producer, topic, 'parsed', data)
        sleep(5)



Begin publish
Message published successfully. Data: 
[{"site_id": "1", "timestamp": "2022-01-01 00:00:00.000", "air_temperature": "3.8", "cloud_coverage": "", "dew_temperature": "2.4", "sea_level_pressure": "1020.9", "wind_direction": "240.0", "wind_speed": "3.1", "weather_ts": "2025-10-28T13:22:35.504868"}, {"site_id": "2", "timestamp": "2022-01-01 00:00:00.000", "air_temperature": "15.6", "cloud_coverage": "6.0", "dew_temperature": "-5.6", "sea_level_pressure": "1015.3", "wind_direction": "270.0", "wind_speed": "3.6", "weather_ts": "2025-10-28T13:22:35.504868"}, {"site_id": "3", "timestamp": "2022-01-01 00:00:00.000", "air_temperature": "10.0", "cloud_coverage": "8.0", "dew_temperature": "2.2", "sea_level_pressure": "1021.1", "wind_direction": "350.0", "wind_speed": "4.1", "weather_ts": "2025-10-28T13:22:35.504868"}, {"site_id": "7", "timestamp": "2022-01-01 00:00:00.000", "air_temperature": "-1.8", "cloud_coverage": "", "dew_temperature": "-3.2", "sea_level_pressure": "1016.0", "wi


Begin publish
Message published successfully. Data: 
[{"site_id": "4", "timestamp": "2022-01-01 08:00:00.000", "air_temperature": "4.4", "cloud_coverage": "0.0", "dew_temperature": "-2.2", "sea_level_pressure": "1020.9", "wind_direction": "0.0", "wind_speed": "0.0", "weather_ts": "2025-10-28T13:22:40.504868"}, {"site_id": "5", "timestamp": "2022-01-01 08:00:00.000", "air_temperature": "5.0", "cloud_coverage": "0.0", "dew_temperature": "4.0", "sea_level_pressure": "", "wind_direction": "110.0", "wind_speed": "7.2", "weather_ts": "2025-10-28T13:22:40.504868"}, {"site_id": "6", "timestamp": "2022-01-01 08:00:00.000", "air_temperature": "5.6", "cloud_coverage": "0.0", "dew_temperature": "2.2", "sea_level_pressure": "1020.1", "wind_direction": "40.0", "wind_speed": "1.5", "weather_ts": "2025-10-28T13:22:40.504868"}, {"site_id": "7", "timestamp": "2022-01-01 08:00:00.000", "air_temperature": "-3.0", "cloud_coverage": "", "dew_temperature": "-3.8", "sea_level_pressure": "1014.1", "wind_direc


Begin publish
Message published successfully. Data: 
[{"site_id": "0", "timestamp": "2022-01-01 15:00:00.000", "air_temperature": "23.9", "cloud_coverage": "", "dew_temperature": "21.1", "sea_level_pressure": "1020.3", "wind_direction": "210.0", "wind_speed": "3.1", "weather_ts": "2025-10-28T13:22:45.504868"}, {"site_id": "1", "timestamp": "2022-01-01 16:00:00.000", "air_temperature": "7.6", "cloud_coverage": "", "dew_temperature": "6.7", "sea_level_pressure": "1013.8", "wind_direction": "110.0", "wind_speed": "5.7", "weather_ts": "2025-10-28T13:22:45.504868"}, {"site_id": "2", "timestamp": "2022-01-01 16:00:00.000", "air_temperature": "5.6", "cloud_coverage": "0.0", "dew_temperature": "-2.8", "sea_level_pressure": "1020.0", "wind_direction": "100.0", "wind_speed": "1.5", "weather_ts": "2025-10-28T13:22:45.504868"}, {"site_id": "3", "timestamp": "2022-01-01 16:00:00.000", "air_temperature": "6.7", "cloud_coverage": "", "dew_temperature": "-1.1", "sea_level_pressure": "1021.4", "wind_d


Begin publish
Message published successfully. Data: 
[{"site_id": "8", "timestamp": "2022-01-01 23:00:00.000", "air_temperature": "25.6", "cloud_coverage": "", "dew_temperature": "18.3", "sea_level_pressure": "1017.5", "wind_direction": "230.0", "wind_speed": "3.1", "weather_ts": "2025-10-28T13:22:50.504868"}, {"site_id": "9", "timestamp": "2022-01-01 23:00:00.000", "air_temperature": "7.2", "cloud_coverage": "", "dew_temperature": "0.6", "sea_level_pressure": "1032.2", "wind_direction": "360.0", "wind_speed": "2.6", "weather_ts": "2025-10-28T13:22:50.504868"}, {"site_id": "10", "timestamp": "2022-01-01 23:00:00.000", "air_temperature": "-8.9", "cloud_coverage": "0.0", "dew_temperature": "-15.0", "sea_level_pressure": "1033.3", "wind_direction": "310.0", "wind_speed": "2.6", "weather_ts": "2025-10-28T13:22:50.504868"}, {"site_id": "11", "timestamp": "2022-01-01 23:00:00.000", "air_temperature": "-2.0", "cloud_coverage": "", "dew_temperature": "-3.2", "sea_level_pressure": "1010.7", "w


Begin publish
Message published successfully. Data: 
[{"site_id": "3", "timestamp": "2022-01-02 07:00:00.000", "air_temperature": "2.2", "cloud_coverage": "0.0", "dew_temperature": "-5.6", "sea_level_pressure": "1020.9", "wind_direction": "220.0", "wind_speed": "2.6", "weather_ts": "2025-10-28T13:22:55.504868"}, {"site_id": "4", "timestamp": "2022-01-02 07:00:00.000", "air_temperature": "5.6", "cloud_coverage": "", "dew_temperature": "-1.1", "sea_level_pressure": "1020.7", "wind_direction": "90.0", "wind_speed": "3.1", "weather_ts": "2025-10-28T13:22:55.504868"}, {"site_id": "5", "timestamp": "2022-01-02 07:00:00.000", "air_temperature": "10.0", "cloud_coverage": "", "dew_temperature": "9.0", "sea_level_pressure": "", "wind_direction": "100.0", "wind_speed": "6.2", "weather_ts": "2025-10-28T13:22:55.504868"}, {"site_id": "6", "timestamp": "2022-01-02 07:00:00.000", "air_temperature": "2.2", "cloud_coverage": "0.0", "dew_temperature": "-6.7", "sea_level_pressure": "1020.3", "wind_direc

...


Begin publish
Message published successfully. Data: 
[{"site_id": "15", "timestamp": "2022-01-13 23:00:00.000", "air_temperature": "-8.3", "cloud_coverage": "4.0", "dew_temperature": "-13.9", "sea_level_pressure": "", "wind_direction": "250.0", "wind_speed": "3.6", "weather_ts": "2025-10-28T13:25:55.504868"}, {"site_id": "0", "timestamp": "2022-01-14 00:00:00.000", "air_temperature": "12.2", "cloud_coverage": "8.0", "dew_temperature": "3.3", "sea_level_pressure": "1021.8", "wind_direction": "360.0", "wind_speed": "4.1", "weather_ts": "2025-10-28T13:25:55.504868"}, {"site_id": "1", "timestamp": "2022-01-14 00:00:00.000", "air_temperature": "4.9", "cloud_coverage": "", "dew_temperature": "4.1", "sea_level_pressure": "1006.4", "wind_direction": "180.0", "wind_speed": "4.1", "weather_ts": "2025-10-28T13:25:55.504868"}, {"site_id": "2", "timestamp": "2022-01-14 00:00:00.000", "air_temperature": "18.3", "cloud_coverage": "4.0", "dew_temperature": "-6.1", "sea_level_pressure": "1016.0", "win


Begin publish
Message published successfully. Data: 
[{"site_id": "7", "timestamp": "2022-01-14 07:00:00.000", "air_temperature": "-14.5", "cloud_coverage": "", "dew_temperature": "-16.1", "sea_level_pressure": "1008.9", "wind_direction": "80.0", "wind_speed": "1.5", "weather_ts": "2025-10-28T13:26:00.504868"}, {"site_id": "8", "timestamp": "2022-01-14 07:00:00.000", "air_temperature": "8.9", "cloud_coverage": "", "dew_temperature": "5.6", "sea_level_pressure": "1021.9", "wind_direction": "360.0", "wind_speed": "2.6", "weather_ts": "2025-10-28T13:26:00.504868"}, {"site_id": "9", "timestamp": "2022-01-14 07:00:00.000", "air_temperature": "8.3", "cloud_coverage": "0.0", "dew_temperature": "4.4", "sea_level_pressure": "1017.4", "wind_direction": "0.0", "wind_speed": "0.0", "weather_ts": "2025-10-28T13:26:00.504868"}, {"site_id": "10", "timestamp": "2022-01-14 07:00:00.000", "air_temperature": "-3.9", "cloud_coverage": "", "dew_temperature": "-6.7", "sea_level_pressure": "1017.0", "wind_d


Begin publish
Message published successfully. Data: 
[{"site_id": "15", "timestamp": "2022-01-14 14:00:00.000", "air_temperature": "-7.2", "cloud_coverage": "", "dew_temperature": "-10.6", "sea_level_pressure": "1011.3", "wind_direction": "200.0", "wind_speed": "3.1", "weather_ts": "2025-10-28T13:26:05.504868"}, {"site_id": "0", "timestamp": "2022-01-14 15:00:00.000", "air_temperature": "13.3", "cloud_coverage": "", "dew_temperature": "5.0", "sea_level_pressure": "1022.5", "wind_direction": "30.0", "wind_speed": "4.1", "weather_ts": "2025-10-28T13:26:05.504868"}, {"site_id": "1", "timestamp": "2022-01-14 15:00:00.000", "air_temperature": "5.0", "cloud_coverage": "", "dew_temperature": "-0.2", "sea_level_pressure": "1007.4", "wind_direction": "290.0", "wind_speed": "7.7", "weather_ts": "2025-10-28T13:26:05.504868"}, {"site_id": "2", "timestamp": "2022-01-14 15:00:00.000", "air_temperature": "6.1", "cloud_coverage": "2.0", "dew_temperature": "0.6", "sea_level_pressure": "1017.1", "wind_


Begin publish
Message published successfully. Data: 
[{"site_id": "7", "timestamp": "2022-01-14 22:00:00.000", "air_temperature": "-8.2", "cloud_coverage": "", "dew_temperature": "-10.9", "sea_level_pressure": "1011.3", "wind_direction": "260.0", "wind_speed": "0.5", "weather_ts": "2025-10-28T13:26:10.504868"}, {"site_id": "8", "timestamp": "2022-01-14 22:00:00.000", "air_temperature": "19.4", "cloud_coverage": "", "dew_temperature": "11.1", "sea_level_pressure": "1018.2", "wind_direction": "70.0", "wind_speed": "3.6", "weather_ts": "2025-10-28T13:26:10.504868"}, {"site_id": "9", "timestamp": "2022-01-14 22:00:00.000", "air_temperature": "21.1", "cloud_coverage": "4.0", "dew_temperature": "12.2", "sea_level_pressure": "1007.8", "wind_direction": "190.0", "wind_speed": "3.6", "weather_ts": "2025-10-28T13:26:10.504868"}, {"site_id": "10", "timestamp": "2022-01-14 22:00:00.000", "air_temperature": "1.1", "cloud_coverage": "", "dew_temperature": "-6.1", "sea_level_pressure": "1017.7", "wi


Begin publish
Message published successfully. Data: 
[{"site_id": "15", "timestamp": "2022-01-15 05:00:00.000", "air_temperature": "-3.9", "cloud_coverage": "", "dew_temperature": "-5.6", "sea_level_pressure": "1012.0", "wind_direction": "80.0", "wind_speed": "1.5", "weather_ts": "2025-10-28T13:26:15.504868"}, {"site_id": "0", "timestamp": "2022-01-15 06:00:00.000", "air_temperature": "16.7", "cloud_coverage": "8.0", "dew_temperature": "14.4", "sea_level_pressure": "1015.2", "wind_direction": "130.0", "wind_speed": "2.6", "weather_ts": "2025-10-28T13:26:15.504868"}, {"site_id": "1", "timestamp": "2022-01-15 06:00:00.000", "air_temperature": "1.1", "cloud_coverage": "", "dew_temperature": "-0.6", "sea_level_pressure": "1017.2", "wind_direction": "290.0", "wind_speed": "2.6", "weather_ts": "2025-10-28T13:26:15.504868"}, {"site_id": "2", "timestamp": "2022-01-15 06:00:00.000", "air_temperature": "10.0", "cloud_coverage": "2.0", "dew_temperature": "2.8", "sea_level_pressure": "1015.9", "w

AI use:
https://chatgpt.com/share/6900d35d-caac-8005-b3c7-ec3854377c2a 